In [1]:
!pip install segmentation-models
!pip install tensorflow==2.8
!pip install keras==2.8


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys

sys.path.append('c:/users/haris/appdata/roaming/python/python39/site-packages')

In [3]:
%matplotlib inline

In [4]:
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [6]:
import tensorflow as tf
import segmentation_models as sm
BACKBONE = 'mobilenetv2'
preprocess_input = sm.get_preprocessing(BACKBONE)

Segmentation Models: using `keras` framework.


In [7]:
sm.set_framework('tf.keras')

sm.framework()

'tf.keras'

In [9]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 224
W = 224

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(dataset_path, split=0.1):
    images = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset/ISIC2018_Training", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset/ISIC2018_Training_Groun_Truth", "*.png")))

    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 32
    lr = 1e-4 ## (0.0001)
    num_epoch = 200
    model_path = "files/model.h5"
    csv_path = "files/data.csv"

    """ Dataset : 80/10/10 """
    dataset_path = "C:/Users/Haris/ICAI_Paper/Dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = sm.Unet(BACKBONE, encoder_weights='imagenet')
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)


    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train: 2076 - 2076
Valid: 259 - 259
Test: 259 - 259
Epoch 1/200
65/65 [==============================] - ETA: 0s - loss: 0.3675 - dice_coef: 0.4993 - iou: 0.3395 - recall_1: 0.6774 - precision_1: 0.7008
Epoch 1: val_loss improved from inf to 2.02422, saving model to files\model.h5
65/65 [==============================] - 344s 5s/step - loss: 0.3675 - dice_coef: 0.4993 - iou: 0.3395 - recall_1: 0.6774 - precision_1: 0.7008 - val_loss: 2.0242 - val_dice_coef: 0.5046 - val_iou: 0.3409 - val_recall_1: 0.9924 - val_precision_1: 0.3500 - lr: 1.0000e-04
Epoch 2/200
65/65 [==============================] - ETA: 0s - loss: 0.1831 - dice_coef: 0.7086 - iou: 0.5504 - recall_1: 0.8871 - precision_1: 0.9012
Epoch 2: val_loss improved from 2.02422 to 0.32290, saving model to files\model.h5
65/65 [==============================] - 312s 5s/step - loss: 0.1831 - dice_coef: 0.7086 - iou: 0.5504 - recall_1: 0.8871 - precision_1: 0.9012 - val_loss: 0.3229 - val_dice_coef: 0.6665 - val_iou: 0.5011 - val_re

Epoch 18/200
65/65 [==============================] - ETA: 0s - loss: 0.0418 - dice_coef: 0.9237 - iou: 0.8584 - recall_1: 0.9701 - precision_1: 0.9758
Epoch 18: val_loss improved from 0.24650 to 0.23588, saving model to files\model.h5
65/65 [==============================] - 311s 5s/step - loss: 0.0418 - dice_coef: 0.9237 - iou: 0.8584 - recall_1: 0.9701 - precision_1: 0.9758 - val_loss: 0.2359 - val_dice_coef: 0.7538 - val_iou: 0.6106 - val_recall_1: 0.7044 - val_precision_1: 0.9708 - lr: 1.0000e-06
Epoch 19/200
65/65 [==============================] - ETA: 0s - loss: 0.0416 - dice_coef: 0.9240 - iou: 0.8590 - recall_1: 0.9704 - precision_1: 0.9759
Epoch 19: val_loss improved from 0.23588 to 0.22684, saving model to files\model.h5
65/65 [==============================] - 311s 5s/step - loss: 0.0416 - dice_coef: 0.9240 - iou: 0.8590 - recall_1: 0.9704 - precision_1: 0.9759 - val_loss: 0.2268 - val_dice_coef: 0.7637 - val_iou: 0.6231 - val_recall_1: 0.7188 - val_precision_1: 0.9691 - l

65/65 [==============================] - ETA: 0s - loss: 0.0355 - dice_coef: 0.9344 - iou: 0.8770 - recall_1: 0.9767 - precision_1: 0.9801
Epoch 50: val_loss improved from 0.16473 to 0.16442, saving model to files\model.h5
65/65 [==============================] - 319s 5s/step - loss: 0.0355 - dice_coef: 0.9344 - iou: 0.8770 - recall_1: 0.9767 - precision_1: 0.9801 - val_loss: 0.1644 - val_dice_coef: 0.8354 - val_iou: 0.7234 - val_recall_1: 0.8485 - val_precision_1: 0.9357 - lr: 1.0000e-06
Epoch 51/200
65/65 [==============================] - ETA: 0s - loss: 0.0353 - dice_coef: 0.9347 - iou: 0.8776 - recall_1: 0.9768 - precision_1: 0.9802
Epoch 51: val_loss improved from 0.16442 to 0.16418, saving model to files\model.h5
65/65 [==============================] - 317s 5s/step - loss: 0.0353 - dice_coef: 0.9347 - iou: 0.8776 - recall_1: 0.9768 - precision_1: 0.9802 - val_loss: 0.1642 - val_dice_coef: 0.8359 - val_iou: 0.7242 - val_recall_1: 0.8494 - val_precision_1: 0.9354 - lr: 1.0000e-06

65/65 [==============================] - 317s 5s/step - loss: 0.0338 - dice_coef: 0.9372 - iou: 0.8820 - recall_1: 0.9781 - precision_1: 0.9814 - val_loss: 0.1621 - val_dice_coef: 0.8410 - val_iou: 0.7321 - val_recall_1: 0.8575 - val_precision_1: 0.9338 - lr: 1.0000e-07
Epoch 67/200
65/65 [==============================] - ETA: 0s - loss: 0.0338 - dice_coef: 0.9372 - iou: 0.8820 - recall_1: 0.9781 - precision_1: 0.9814
Epoch 67: val_loss improved from 0.16211 to 0.16197, saving model to files\model.h5
65/65 [==============================] - 310s 5s/step - loss: 0.0338 - dice_coef: 0.9372 - iou: 0.8820 - recall_1: 0.9781 - precision_1: 0.9814 - val_loss: 0.1620 - val_dice_coef: 0.8412 - val_iou: 0.7324 - val_recall_1: 0.8579 - val_precision_1: 0.9337 - lr: 1.0000e-07
Epoch 68/200
65/65 [==============================] - ETA: 0s - loss: 0.0337 - dice_coef: 0.9372 - iou: 0.8821 - recall_1: 0.9782 - precision_1: 0.9814
Epoch 68: val_loss improved from 0.16197 to 0.16188, saving model to f

Epoch 83/200
65/65 [==============================] - ETA: 0s - loss: 0.0334 - dice_coef: 0.9378 - iou: 0.8830 - recall_1: 0.9785 - precision_1: 0.9816
Epoch 83: val_loss improved from 0.16084 to 0.16082, saving model to files\model.h5
65/65 [==============================] - 309s 5s/step - loss: 0.0334 - dice_coef: 0.9378 - iou: 0.8830 - recall_1: 0.9785 - precision_1: 0.9816 - val_loss: 0.1608 - val_dice_coef: 0.8430 - val_iou: 0.7350 - val_recall_1: 0.8610 - val_precision_1: 0.9326 - lr: 1.0000e-07
Epoch 84/200
65/65 [==============================] - ETA: 0s - loss: 0.0334 - dice_coef: 0.9378 - iou: 0.8831 - recall_1: 0.9785 - precision_1: 0.9816
Epoch 84: val_loss improved from 0.16082 to 0.16081, saving model to files\model.h5
65/65 [==============================] - 307s 5s/step - loss: 0.0334 - dice_coef: 0.9378 - iou: 0.8831 - recall_1: 0.9785 - precision_1: 0.9816 - val_loss: 0.1608 - val_dice_coef: 0.8431 - val_iou: 0.7350 - val_recall_1: 0.8611 - val_precision_1: 0.9325 - l

Epoch 100/200
65/65 [==============================] - ETA: 0s - loss: 0.0331 - dice_coef: 0.9384 - iou: 0.8842 - recall_1: 0.9788 - precision_1: 0.9818
Epoch 100: val_loss did not improve from 0.16075
65/65 [==============================] - 322s 5s/step - loss: 0.0331 - dice_coef: 0.9384 - iou: 0.8842 - recall_1: 0.9788 - precision_1: 0.9818 - val_loss: 0.1609 - val_dice_coef: 0.8439 - val_iou: 0.7362 - val_recall_1: 0.8622 - val_precision_1: 0.9320 - lr: 1.0000e-07
Epoch 101/200
65/65 [==============================] - ETA: 0s - loss: 0.0331 - dice_coef: 0.9385 - iou: 0.8842 - recall_1: 0.9788 - precision_1: 0.9818
Epoch 101: val_loss did not improve from 0.16075
65/65 [==============================] - 323s 5s/step - loss: 0.0331 - dice_coef: 0.9385 - iou: 0.8842 - recall_1: 0.9788 - precision_1: 0.9818 - val_loss: 0.1609 - val_dice_coef: 0.8440 - val_iou: 0.7363 - val_recall_1: 0.8622 - val_precision_1: 0.9320 - lr: 1.0000e-07
Epoch 102/200
65/65 [==============================] -

In [10]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_2[0][0]']                
                                3)                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, None, None,   864         ['Conv1_pad[0][0]']              
                                32)                                                         

                                                                                                  
 block_2_add (Add)              (None, None, None,   0           ['block_1_project_BN[0][0]',     
                                24)                               'block_2_project_BN[0][0]']     
                                                                                                  
 block_3_expand (Conv2D)        (None, None, None,   3456        ['block_2_add[0][0]']            
                                144)                                                              
                                                                                                  
 block_3_expand_BN (BatchNormal  (None, None, None,   576        ['block_3_expand[0][0]']         
 ization)                       144)                                                              
                                                                                                  
 block_3_e

                                32)                               'block_5_project_BN[0][0]']     
                                                                                                  
 block_6_expand (Conv2D)        (None, None, None,   6144        ['block_5_add[0][0]']            
                                192)                                                              
                                                                                                  
 block_6_expand_BN (BatchNormal  (None, None, None,   768        ['block_6_expand[0][0]']         
 ization)                       192)                                                              
                                                                                                  
 block_6_expand_relu (ReLU)     (None, None, None,   0           ['block_6_expand_BN[0][0]']      
                                192)                                                              
          

 block_9_expand (Conv2D)        (None, None, None,   24576       ['block_8_add[0][0]']            
                                384)                                                              
                                                                                                  
 block_9_expand_BN (BatchNormal  (None, None, None,   1536       ['block_9_expand[0][0]']         
 ization)                       384)                                                              
                                                                                                  
 block_9_expand_relu (ReLU)     (None, None, None,   0           ['block_9_expand_BN[0][0]']      
                                384)                                                              
                                                                                                  
 block_9_depthwise (DepthwiseCo  (None, None, None,   3456       ['block_9_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_12_expand_relu (ReLU)    (None, None, None,   0           ['block_12_expand_BN[0][0]']     
                                576)                                                              
                                                                                                  
 block_12_depthwise (DepthwiseC  (None, None, None,   5184       ['block_12_expand_relu[0][0]']   
 onv2D)                         576)                                                              
                                                                                                  
 block_12_depthwise_BN (BatchNo  (None, None, None,   2304       ['block_12_depthwise[0][0]']     
 rmalization)                   576)                                                              
                                                                                                  
 block_12_

                                960)                                                              
                                                                                                  
 block_15_depthwise (DepthwiseC  (None, None, None,   8640       ['block_15_expand_relu[0][0]']   
 onv2D)                         960)                                                              
                                                                                                  
 block_15_depthwise_BN (BatchNo  (None, None, None,   3840       ['block_15_depthwise[0][0]']     
 rmalization)                   960)                                                              
                                                                                                  
 block_15_depthwise_relu (ReLU)  (None, None, None,   0          ['block_15_depthwise_BN[0][0]']  
                                960)                                                              
          

                                                                  'block_6_expand_relu[0][0]']    
                                                                                                  
 decoder_stage1a_conv (Conv2D)  (None, None, None,   516096      ['decoder_stage1_concat[0][0]']  
                                128)                                                              
                                                                                                  
 decoder_stage1a_bn (BatchNorma  (None, None, None,   512        ['decoder_stage1a_conv[0][0]']   
 lization)                      128)                                                              
                                                                                                  
 decoder_stage1a_relu (Activati  (None, None, None,   0          ['decoder_stage1a_bn[0][0]']     
 on)                            128)                                                              
          

                                16)                                                               
                                                                                                  
 decoder_stage4b_bn (BatchNorma  (None, None, None,   64         ['decoder_stage4b_conv[0][0]']   
 lization)                      16)                                                               
                                                                                                  
 decoder_stage4b_relu (Activati  (None, None, None,   0          ['decoder_stage4b_bn[0][0]']     
 on)                            16)                                                               
                                                                                                  
 final_conv (Conv2D)            (None, None, None,   145         ['decoder_stage4b_relu[0][0]']   
                                1)                                                                
          

In [11]:
tf.keras.utils.plot_model(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [15]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score



H = 256
W = 256

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                ## (1, 256, 256, 3)


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                    ## (256, 256)
    return ori_x, x

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)  ## (256, 256, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) ## (256, 256, 3)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving results """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")

    """ Load the test data """
    dataset_path = "C:/Users/Haris/ICAI_Paper"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Exctracting the image name """
        name = x.split("/")[-1]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Predicting the mask """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        """ Saving the predicted mask """
        save_image_path = f"results/{name}"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ mean metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

100%|████████████████████████████████████████████████████████████████████████████████| 259/259 [01:11<00:00,  3.63it/s]

Accuracy: 0.95758
F1: 0.88633
Jaccard: 0.80877
Recall: 0.90138
Precision: 0.89768


In [8]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    """ Encoder """
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    """ Bridge """
    b1 = conv_block(p4, 1024)

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    """ Outputs """
    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    """ Model """
    model = Model(inputs, outputs)
    return model

In [10]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision


H = 224
W = 224

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def load_data(dataset_path, split=0.1):
    images = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset/ISIC2018_Training", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset/ISIC2018_Training_Groun_Truth", "*.png")))

    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files")

    """ Hyperparameters """
    batch_size = 16
    lr = 1e-4 ## (0.0001)
    num_epoch = 200
    model_path = "files/model.h5"
    csv_path = "files/data.csv"

    """ Dataset : 80/10/10 """
    dataset_path = "C:/Users/Haris/ICAI_Paper/Dataset"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    """ Model """
    model = build_unet((H, W, 3))
    metrics = [dice_coef, iou, Recall(), Precision()]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)


    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=valid_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )

Train: 2076 - 2076
Valid: 259 - 259
Test: 259 - 259
Epoch 1/200
130/130 [==============================] - ETA: 0s - loss: 0.3143 - dice_coef: 0.5800 - iou: 0.4135 - recall_1: 0.6631 - precision_1: 0.7455
Epoch 1: val_loss improved from inf to 0.53061, saving model to files\model.h5
130/130 [==============================] - 344s 3s/step - loss: 0.3143 - dice_coef: 0.5800 - iou: 0.4135 - recall_1: 0.6631 - precision_1: 0.7455 - val_loss: 0.5306 - val_dice_coef: 0.2563 - val_iou: 0.1472 - val_recall_1: 0.0428 - val_precision_1: 0.9943 - lr: 1.0000e-04
Epoch 2/200
130/130 [==============================] - ETA: 0s - loss: 0.2121 - dice_coef: 0.6931 - iou: 0.5325 - recall_1: 0.7572 - precision_1: 0.8557
Epoch 2: val_loss improved from 0.53061 to 0.51358, saving model to files\model.h5
130/130 [==============================] - 321s 2s/step - loss: 0.2121 - dice_coef: 0.6931 - iou: 0.5325 - recall_1: 0.7572 - precision_1: 0.8557 - val_loss: 0.5136 - val_dice_coef: 0.3804 - val_iou: 0.2385 

Epoch 18/200
130/130 [==============================] - ETA: 0s - loss: 0.0833 - dice_coef: 0.8774 - iou: 0.7823 - recall_1: 0.9072 - precision_1: 0.9355
Epoch 18: val_loss did not improve from 0.14566

Epoch 18: ReduceLROnPlateau reducing learning rate to 9.999999747378752e-06.
130/130 [==============================] - 310s 2s/step - loss: 0.0833 - dice_coef: 0.8774 - iou: 0.7823 - recall_1: 0.9072 - precision_1: 0.9355 - val_loss: 0.1637 - val_dice_coef: 0.8275 - val_iou: 0.7081 - val_recall_1: 0.8030 - val_precision_1: 0.9532 - lr: 1.0000e-04
Epoch 19/200
130/130 [==============================] - ETA: 0s - loss: 0.0727 - dice_coef: 0.8912 - iou: 0.8044 - recall_1: 0.9223 - precision_1: 0.9400
Epoch 19: val_loss improved from 0.14566 to 0.12983, saving model to files\model.h5
130/130 [==============================] - 310s 2s/step - loss: 0.0727 - dice_coef: 0.8912 - iou: 0.8044 - recall_1: 0.9223 - precision_1: 0.9400 - val_loss: 0.1298 - val_dice_coef: 0.8572 - val_iou: 0.7516 - 

Epoch 35/200
130/130 [==============================] - ETA: 0s - loss: 0.0506 - dice_coef: 0.9208 - iou: 0.8535 - recall_1: 0.9463 - precision_1: 0.9594
Epoch 35: val_loss did not improve from 0.12753
130/130 [==============================] - 311s 2s/step - loss: 0.0506 - dice_coef: 0.9208 - iou: 0.8535 - recall_1: 0.9463 - precision_1: 0.9594 - val_loss: 0.1374 - val_dice_coef: 0.8730 - val_iou: 0.7760 - val_recall_1: 0.8728 - val_precision_1: 0.9289 - lr: 1.0000e-07
Epoch 36/200
130/130 [==============================] - ETA: 0s - loss: 0.0505 - dice_coef: 0.9208 - iou: 0.8536 - recall_1: 0.9465 - precision_1: 0.9593
Epoch 36: val_loss did not improve from 0.12753
130/130 [==============================] - 311s 2s/step - loss: 0.0505 - dice_coef: 0.9208 - iou: 0.8536 - recall_1: 0.9465 - precision_1: 0.9593 - val_loss: 0.1374 - val_dice_coef: 0.8730 - val_iou: 0.7760 - val_recall_1: 0.8731 - val_precision_1: 0.9287 - lr: 1.0000e-07
Epoch 37/200
130/130 [============================

In [11]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
import pandas as pd
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score



H = 224
W = 224

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                ## (1, 256, 256, 3)


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                    ## (256, 256)
    return ori_x, x

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)  ## (256, 256, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) ## (256, 256, 3)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving results """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")

    """ Load the test data """
    dataset_path = "/content/drive/MyDrive/"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Exctracting the image name """
        name = x.split("/")[-1]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Predicting the mask """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        """ Saving the predicted mask """
        save_image_path = f"results/{name}"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ mean metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

 69%|███████████████████████████████████████████████████████▌                        | 180/259 [00:53<00:11,  6.94it/s]C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
 97%|█████████████████████████████████████████████████████████████████████████████▊  | 252/259 [01:13<00:01,  4.46it/s]C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|████████████████████████████████████████████████████████████████████████████████| 259/259 [01:14<00:00,  3.47it/s]

Accuracy: 0.95681
F1: 0.88410
Jaccard: 0.81170
Recall: 0.91029
Precision: 0.88815


In [12]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_19 (Conv2D)             (None, 224, 224, 64  1792        ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_18 (BatchN  (None, 224, 224, 64  256        ['conv2d_19[0][0]']              
 ormalization)                  )                                                           

 ormalization)                  )                                                                 
                                                                                                  
 activation_26 (Activation)     (None, 14, 14, 1024  0           ['batch_normalization_26[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_28 (Conv2D)             (None, 14, 14, 1024  9438208     ['activation_26[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_27 (BatchN  (None, 14, 14, 1024  4096       ['conv2d_28[0][0]']              
 ormalization)                  )                                                                 
          

                                                                                                  
 conv2d_35 (Conv2D)             (None, 224, 224, 64  73792       ['concatenate_7[0][0]']          
                                )                                                                 
                                                                                                  
 batch_normalization_34 (BatchN  (None, 224, 224, 64  256        ['conv2d_35[0][0]']              
 ormalization)                  )                                                                 
                                                                                                  
 activation_34 (Activation)     (None, 224, 224, 64  0           ['batch_normalization_34[0][0]'] 
                                )                                                                 
                                                                                                  
 conv2d_36

In [3]:
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import pandas as pd
from glob import glob
from tqdm import tqdm
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score

import sys

sys.path.append('c:/users/haris/appdata/roaming/python/python39/site-packages')
import cv2
H = 224
W = 224
IMAGE_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 2


def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)


def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y


def load_data(dataset_path, split=0.1):
    images = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset/ISIC2018_Training", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "C:/Users/Haris/ICAI_Paper/Dataset/ISIC2018_Training_Groun_Truth", "*.png")))

    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ##(H, W, 3)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)
    return x  ##(256, 256, 3)


def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ##(H, W)
    x = cv2.resize(x, (W, H))
    x = x / 255.0
    x = x.astype(np.float32)  ##(256, 256)
    x = np.expand_dims(x, axis=-1)  ##(256, 256, 1)
    return x


def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y


def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)
    return dataset

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)


def convolution_block(
        block_input,
        num_filters=256,
        kernel_size=3,
        dilation_rate=1,
        padding="same",
        use_bias=False,
):
    x = layers.Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
        kernel_initializer=keras.initializers.HeNormal(),
    )(block_input)
    x = layers.BatchNormalization()(x)
    return tf.nn.relu(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = layers.AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = layers.UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=1)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=6)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=18)

    x = layers.Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output


def DeeplabV3Plus(image_size, num_classes):
    model_input = keras.Input(shape=(image_size, image_size, 3))
    resnet50 = keras.applications.ResNet50(
        weights="imagenet", include_top=False, input_tensor=model_input
    )
    x = resnet50.get_layer("conv4_block6_2_relu").output
    x = DilatedSpatialPyramidPooling(x)

    input_a = layers.UpSampling2D(
        size=(image_size // 4 // x.shape[1], image_size // 4 // x.shape[2]),
        interpolation="bilinear",
    )(x)
    input_b = resnet50.get_layer("conv2_block3_2_relu").output
    input_b = convolution_block(input_b, num_filters=48, kernel_size=1)

    x = layers.Concatenate(axis=-1)([input_a, input_b])
    x = convolution_block(x)
    x = convolution_block(x)
    x = layers.UpSampling2D(
        size=(image_size // x.shape[1], image_size // x.shape[2]),
        interpolation="bilinear",
    )(x)
    model_output = layers.Conv2D(num_classes, kernel_size=(1, 1), padding="same")(x)
    return keras.Model(inputs=model_input, outputs=model_output)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("files")

    """ Hyperparameters """

    batch_size = 32
    lr = 1e-4  ##(0.0001)
    num_epoch = 100
    model_path = "files/model.h5"
    csv_path = "files/data.csv"

    """ Dataset : 60/20/20 """
    dataset_path = "C:/Users/Haris/ICAI_Paper/Dataset/"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    val_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x) // batch_size
    valid_steps = len(valid_x) // batch_size

    if len(train_x) % batch_size != 0:
        train_steps += 1

    if len(valid_x) % batch_size != 0:
        valid_steps += 1

    print("Train Dataset:", train_dataset)
    print("Val Dataset:", val_dataset)


    model = DeeplabV3Plus(image_size=IMAGE_SIZE, num_classes=NUM_CLASSES)
    model.summary()

    metrics = [dice_coef, iou, Recall(), Precision(), "accuracy"]
    model.compile(loss="binary_crossentropy", optimizer=Adam(lr), metrics=metrics)

    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=False)
    ]

    history = model.fit(
        train_dataset,
        epochs=num_epoch,
        validation_data=val_dataset,
        steps_per_epoch=train_steps,
        validation_steps=valid_steps,
        callbacks=callbacks
    )



    plt.plot(history.history["loss"])
    plt.title("Training Loss")
    plt.ylabel("loss")
    plt.xlabel("epoch")
    plt.show()

    plt.plot(history.history["accuracy"])
    plt.title("Training Accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.show()

    plt.plot(history.history["val_loss"])
    plt.title("Validation Loss")
    plt.ylabel("val_loss")
    plt.xlabel("epoch")
    plt.show()

    plt.plot(history.history["val_accuracy"])
    plt.title("Validation Accuracy")
    plt.ylabel("val_accuracy")
    plt.xlabel("epoch")
    plt.show()


def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=0)
    return ori_x, x                                ## (1, 256, 256, 3)


def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.int32)                    ## (256, 256)
    return ori_x, x

def save_results(ori_x, ori_y, y_pred, save_image_path):
    line = np.ones((H, 10, 3)) * 255

    ori_y = np.expand_dims(ori_y, axis=-1)  ## (256, 256, 1)
    ori_y = np.concatenate([ori_y, ori_y, ori_y], axis=-1) ## (256, 256, 3)

    y_pred = np.expand_dims(y_pred, axis=-1)  ## (256, 256, 1)
    y_pred = np.concatenate([y_pred, y_pred, y_pred], axis=-1) ## (256, 256, 3)

    cat_images = np.concatenate([ori_x, line, ori_y, line, y_pred*255], axis=1)
    cv2.imwrite(save_image_path, cat_images)


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving results """
    create_dir("results")

    """ Load the model """
    with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef}):
        model = tf.keras.models.load_model("files/model.h5")

    """ Load the test data """
    dataset_path = "/content/drive/MyDrive/"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    SCORE = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Exctracting the image name """
        name = x.split("/")[-1]

        """ Read the image and mask """
        ori_x, x = read_image(x)
        ori_y, y = read_mask(y)

        """ Predicting the mask """
        y_pred = model.predict(x)[0] > 0.5
        y_pred = np.squeeze(y_pred, axis=-1)
        y_pred = y_pred.astype(np.int32)

        """ Saving the predicted mask """
        save_image_path = f"results/{name}"
        save_results(ori_x, ori_y, y_pred, save_image_path)

        """ Flatten the array """
        y = y.flatten()
        y_pred = y_pred.flatten()

        """ Calculating metrics values """
        acc_value = accuracy_score(y, y_pred)
        f1_value = f1_score(y, y_pred, labels=[0, 1], average="binary")
        jac_value = jaccard_score(y, y_pred, labels=[0, 1], average="binary")
        recall_value = recall_score(y, y_pred, labels=[0, 1], average="binary")
        precision_value = precision_score(y, y_pred, labels=[0, 1], average="binary")
        SCORE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value])

    """ mean metrics values """
    score = [s[1:] for s in SCORE]
    score = np.mean(score, axis=0)
    print(f"Accuracy: {score[0]:0.5f}")
    print(f"F1: {score[1]:0.5f}")
    print(f"Jaccard: {score[2]:0.5f}")
    print(f"Recall: {score[3]:0.5f}")
    print(f"Precision: {score[4]:0.5f}")

    df = pd.DataFrame(SCORE, columns = ["Image Name", "Acc", "F1", "Jaccard", "Recall", "Precision"])
    df.to_csv("files/score.csv")

Train: 2076 - 2076
Valid: 259 - 259
Test: 259 - 259
Train Dataset: <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None))>
Val Dataset: <PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 224, 224, 1), dtype=tf.float32, name=None))>
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input

 conv2_block3_1_conv (Conv2D)   (None, 56, 56, 64)   16448       ['conv2_block2_out[0][0]']       
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_blo

 conv3_block3_1_conv (Conv2D)   (None, 28, 28, 128)  65664       ['conv3_block2_out[0][0]']       
                                                                                                  
 conv3_block3_1_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block1_out (Activation)  (None, 14, 14, 1024  0           ['conv4_block1_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block2_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block1_out[0][0]']       
                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv4_block4_out (Activation)  (None, 14, 14, 1024  0           ['conv4_block4_add[0][0]']       
                                )                                                                 
                                                                                                  
 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)       

                                                                                                  
 tf.nn.relu_12 (TFOpLambda)     (None, 14, 14, 256)  0           ['batch_normalization_12[0][0]'] 
                                                                                                  
 tf.nn.relu_13 (TFOpLambda)     (None, 14, 14, 256)  0           ['batch_normalization_13[0][0]'] 
                                                                                                  
 concatenate_2 (Concatenate)    (None, 14, 14, 1280  0           ['up_sampling2d_3[0][0]',        
                                )                                 'tf.nn.relu_10[0][0]',          
                                                                  'tf.nn.relu_11[0][0]',          
                                                                  'tf.nn.relu_12[0][0]',          
                                                                  'tf.nn.relu_13[0][0]']          
          

ValueError: in user code:

    File "C:\Users\Haris\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1021, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Haris\AppData\Local\Temp\ipykernel_9008\2664358455.py", line 99, in dice_coef  *
        intersection = tf.reduce_sum(y_true * y_pred)

    ValueError: Dimensions must be equal, but are 50176 and 100352 for '{{node mul_1}} = Mul[T=DT_FLOAT](flatten/Reshape, flatten_1/Reshape)' with input shapes: [?,50176], [?,100352].
